### Import libraries

In [ ]:
import pickle
import pandas as pd
import numpy as np

### Load resources

In [ ]:
# load the model from disk
filename = "finalized_model.sav"
model = pickle.load(open(filename, 'rb'))

### Predict by each condition

In [ ]:
# define variables
cols_when_model_builds
conditions = ["AL", "BL", "AL+CD"]
methods = ["AL", "AL+CD", "BL"] 
results = []
i = 0

for condition in conditions:
    print("------- condition ", condition , "-------")
    
    users = pd.read_csv("../methods.csv")
    users = users[users["method"]==condition]

    method = users["method"].values[0]
    
    for user in users["user_id"].values:
        df = pd.read_csv("../users-feats/train_data_imwut20_"+str(user)+".csv")
        df.dropna()

        X = df[df.columns[~df.columns.isin(['Unnamed: 0','act','user_id','time0_start','time1_finish','time','name'])]]
        X = X.fillna(0)
        X = X[cols_when_model_builds]
        
        y_pred = model.predict(X)
        
        from collections import Counter
        counter = Counter(y_pred)
        
        for k, v in counter.items():
            per = v/len(y_pred)
            
            d = {'method': methods[i], 'user_id': user,'class': k, 'count': v, 'percentage': per}
            results.append(d)
        
    i+=1

### Export results

In [ ]:
df = pd.DataFrame(data=results)
df["class"].replace({1: "Accurate", 0: "Inaccurate"}, inplace=True)
df.to_csv('data/predict_inaccuracies.csv')